**Assignment-2-IR-part-1**

**Importing library**

In [ ]:
import pandas as pd
import numpy as np
import cv2
import requests
import os
import pickle
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

**connecting googgle drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Load Dataset**

In [ ]:
csv_file = "/content/drive/MyDrive/A2_Data.csv"  # Change this to the path of your CSV file
data = pd.read_csv(csv_file)

In [ ]:
print(data)

preprocessing of image using vgg16

In [ ]:
def preprocess_image_from_url(image_url):
    try:
        # Remove any extra characters like single quotes, double quotes, and spaces
        image_url = image_url.strip("[]\"' ")

        # Check if the URL starts with 'http://' or 'https://'
        if not image_url.startswith("http://") and not image_url.startswith("https://"):
            image_url = "http://" + image_url  # Add 'http://' if missing

        print("Downloading image from:", image_url)  # Print the URL for debugging

        # Send a GET request to download the image
        response = requests.get(image_url)
        if response.status_code == 200:
            # Decode the content of the image
            image_data = response.content
            # Convert image data to numpy array
            nparr = np.frombuffer(image_data, np.uint8)
            # Read image from numpy array
            image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

            # Resize image to the input size of VGG16 (224x224)
            resized_image = cv2.resize(image, (224, 224))

            # # Convert BGR to RGB (VGG16 expects RGB images)
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

            # # Normalize pixel values to range [0, 1]
            resized_image = resized_image.astype("float32") / 255.0

            # Apply preprocess_input from VGG16
            preprocessed_image = preprocess_input(resized_image)

            return preprocessed_image
        else:
            print("Failed to download image from:", image_url)
            return None
    except Exception as e:
        print("Error:", e)
        return None


# Path to the CSV file containing the dataset
csv_file = "/content/drive/MyDrive/A2_Data.csv"

# Read CSV file
data = pd.read_csv(csv_file)

# Define an empty dictionary to store the data
data_dict = {}

# Load pre-trained VGG16 model without the top classification layer
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a model with VGG16 up to a certain layer
feature_extractor = Model(inputs=vgg16.input, outputs=vgg16.get_layer('block5_conv2').output)

# Process each row in the dataset
for index, row in data.iterrows():
    # Extract image URLs from the 'Image' column
    image_urls_str = row['Image']
    # Split the string by comma to extract individual URLs
    image_urls = [url.strip("[]\"'") for url in image_urls_str.strip("[]").replace("'", "").split(", ")]
    # Process each image URL
    for image_url in image_urls:
        # Preprocess the image
        preprocessed_image = preprocess_image_from_url(image_url)
        if preprocessed_image is not None:
            # Expand dimensions to match the input shape of VGG16
            preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
            # Extract features using VGG16 model
            features = feature_extractor.predict(preprocessed_image)
            # Flatten the features
            features = features.flatten()
            # Store the image URL and its corresponding feature vector in the dictionary
            data_dict[image_url] = features

In [ ]:
print(data_dict)

In [ ]:
print(len(data_dict))

1610


In [ ]:
pickle_file_path = "/content/image_feature.pickle"

# Open the file in binary write mode
with open(pickle_file_path, "wb") as pickle_file:
    # Dump the dictionary into the pickle file
    pickle.dump(data_dict, pickle_file)
print("Data has been saved to", pickle_file_path)